In [1]:
import json
import keyword
import unicodedata
from collections import defaultdict
import pandas as pd
from extract import SocialETL, construct_query_for_twarc, extract_tags, SocialDB
from rich import print
from rich.console import Console
from rich.progress import track
from rich.table import Table
#import warnings 
#warnings.filterwarnings('always')   #suppresses all warnings
#warnings.filterwarnings('ignore')

## 1 parte: categorizzazione hashtags per prorus, proukr, pax

In [2]:
def ensure_latin(s):
    return (
        unicodedata.normalize("NFKD", s).encode("latin-1", "ignore").decode("latin-1")
    )


def construct_query_for_pandas(root_tags: list) -> str:
    my_query = " or ".join(root_tags)
    return my_query


def create_score(df: pd.DataFrame, one_hashtag: str, root_tags: dict) -> list:
    # takes a df of all tweets, each row is a tweet, each column is
    # a hashtag either [True | False] depending on whether the hashtag
    # is in the tweet. Takes one_hashtag, the hashtag to score, and
    # root_tags, a dict of {category: [hashtags]}
    # TODO: make it so it accepts a list of {category: [hashtags]} instead of just one
    threshold_support = 0.5 * len(df) / 10000  # set the threshold_support here!!

    if threshold_support < 1:
        threshold_support = 1
    # print(f"Looking for {one_hashtag} with support {threshold_support}…")
    mask = df[one_hashtag]
    df = df[mask]
    # print(f"I found {len(df)} tweets with {one_hashtag}")
    if len(df) < threshold_support:
        return False
    """for col in df:
        if df[col].any():
            print(f"We have {col} hashtag")"""
    results = []
    # START OF TESTING:
    """    print(df.info())
    print(df.describe())
    print(df.head())
    print(f"[purple]Dude, here's the breakdown:")
    for tag in root_tags.values():
        print(f"[blue]This is {tag[0]}:")
        print(df[tag[0]])"""
    for category, tag_madre in root_tags.items():
        try:
            my_query = construct_query_for_pandas(tag_madre)
            # print(f"The pandas query is: [red]{my_query}")
            temp_df = df.query(my_query)
            # print(f"Holy jezuz: {len(temp_df)} and {len(df)}")
            results.append(round(len(temp_df) / len(df), 4))
        except KeyError:
            results.append(0)

    # print(results)
    return (results, len(df))


def do_search(tagmadre, pages):
    # construct the initial query to Twarc
    query_madre = construct_query_for_twarc(tagmadre)
    m = SocialETL(
        query=f"({query_madre})",
        pages=pages,
        recent=False,
    )

    # dropping any tweets with no hashtags (I think)
    tweets_with_hashtag = m.df[["id", "entities.hashtags"]].dropna()
    print(
        f"{len(m.df)} tweets retrieved\nwith query '{query_madre}'\nof which {len(tweets_with_hashtag)} tweets with at least 1 hashtag."
    )

    # evaluate the string in "entities.hashtags" to an actual list of dicts
    tweets_with_hashtag["entities.hashtags"] = tweets_with_hashtag[
        "entities.hashtags"
    ].map(eval)

    # make a simple list of strings, one hashtag is one string, into column "tags"
    tweets_with_hashtag["tags"] = tweets_with_hashtag["entities.hashtags"].map(
        extract_tags
    )
    tweets_with_hashtag = tweets_with_hashtag.drop(columns="entities.hashtags")

    # hashtags: EXPLODE *musica dei power ranger*
    all_hashtags = set(tweets_with_hashtag["tags"].explode())

    # keyword level: eliminate all python keywords from columns because otherwise we are in no man's land
    keywordsss = keyword.kwlist
    keywordsss.extend(keyword.softkwlist)
    all_hashtags = all_hashtags.difference(keywordsss)
    all_hashtags = all_hashtags.difference(set(("",)))

    print(f"We have {len(all_hashtags)} unique hashtags.")

    tweets_with_hashtag.set_index("id", inplace=True)

    col_h = sorted(list(all_hashtags))
    df_h = pd.DataFrame(columns=col_h)
    tweets_with_hashtag = pd.concat([tweets_with_hashtag, df_h], axis=1)
    tweets_with_hashtag = tweets_with_hashtag.fillna(False)

    def assign_hashtag_to_tweet(row: pd.Series) -> pd.Series:
        for tag in row["tags"]:
            if tag in all_hashtags:
                row.loc[tag] = True
        return row

    tweets_with_hashtag = tweets_with_hashtag.apply(assign_hashtag_to_tweet, axis=1)
    tweets_with_hashtag = tweets_with_hashtag.drop(columns=["tags"])

    for madre in tagmadre.values():
        print(f"[red]Let's write the describe of column '{madre[0]}'")
        print(tweets_with_hashtag[madre[0]].describe())

    tweets_with_hashtag.to_csv(f"twee_hash_{pages}.csv")
    print("[green]All done.")


if __name__ == "__main__":
    # params
    top_results_to_take = 3
    pages_to_do = 10000

    # set the initial "parent" hashtags for each category
    tag_madre = {
        "proukr": ["slavaukraini"],
        "prorus": ["istandwithputin"],
        "pax": ["stopwarinukraine"],
    }

    # code
    # first run
    do_search(tag_madre, pages_to_do)


Output()

rate limit exceeded: sleeping 62.1423225402832 secs
rate limit exceeded: sleeping 67.4985761642456 secs
rate limit exceeded: sleeping 62.186952114105225 secs
rate limit exceeded: sleeping 71.96235036849976 secs
rate limit exceeded: sleeping 87.73055481910706 secs
rate limit exceeded: sleeping 77.09677290916443 secs
rate limit exceeded: sleeping 73.452321767807 secs
rate limit exceeded: sleeping 64.49164414405823 secs
rate limit exceeded: sleeping 77.71235728263855 secs
rate limit exceeded: sleeping 68.8911452293396 secs
rate limit exceeded: sleeping 110.91895866394043 secs
rate limit exceeded: sleeping 116.71228313446045 secs
rate limit exceeded: sleeping 112.2099621295929 secs
rate limit exceeded: sleeping 81.93051266670227 secs
rate limit exceeded: sleeping 71.5556812286377 secs
rate limit exceeded: sleeping 84.66341042518616 secs
rate limit exceeded: sleeping 76.75438499450684 secs
rate limit exceeded: sleeping 68.89814400672913 secs
rate limit exceeded: sleeping 54.32510471343994 s

MemoryError: Unable to allocate 334. GiB for an array with shape (45728, 980317) and data type object

## 2 parte: scaricare df e distinzione 4 cat (no care) dalle altre

In [14]:
m=SocialDB(placeholder=True)

In [9]:
#modifiche: cambiare data e togliere lang eng
m = SocialETL(query="(#eurovision OR #covid OR #pridemonth OR #grammys OR #justiceforjohnnydepp OR #oscars OR #elonmusk OR #putin OR #russia OR #onepiece OR #ukraine OR #war)", pages=1, recent=False) #strangerthings4 OR onepiece

Output()

In [15]:
m.df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 586581 to 980024
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   class   100 non-null    category
dtypes: category(1)
memory usage: 1.1 KB


In [16]:
m.df['author.verified'].value_counts()

KeyError: 'author.verified'

In [8]:
m.df['author.location'].isna().value_counts()

KeyError: 'author.location'

In [23]:
m.df['geo.coordinates.coordinates'].isna().value_counts()

True     19976
False       22
Name: geo.coordinates.coordinates, dtype: int64

In [24]:
m.df['geo.country_code'].isna().value_counts()

True     19909
False       89
Name: geo.country_code, dtype: int64

In [ ]:
m.df['']

In [13]:
user_id=m.df['author_id']
set_user=set(user_id)
#un utente può aver fatto più di un tweet, in questo modo andiamo a vedere i suoi ultimi tweet solo una volta.
print(len(set_user))

In [5]:
#estarre pure retweeted_user_id per edge
hashtag_for_user={}
retweet_for_user={}
verified_for_user={}
location_for_user={}
#name_for_user={}
#username_for_user={}

for e in set_user:                                                             #1 pages=20 tweet   
    u=UserETL(e,pages=5)                                                       #aggiungere più pagine (1pag=20)
    #preprocessing hashtag
    u.df=u.df.dropna(subset=['entities.hashtags'])                             #drop nan values on column hashtags
    u.df.reset_index()
    u.df['entities.hashtags']=u.df['entities.hashtags'].apply(eval)            #convert into dict
    u.df["entities.hashtags"] = u.df["entities.hashtags"].map(h.extract_tags)  #extract tags
    hashtag_for_user[e]=u.df["entities.hashtags"].tolist()                     #convert pandas series into a list
    #preprocessing retweet
    u.df=u.df.dropna(subset=['retweeted_user_id'])
    retweet_for_user[e]=u.df['retweeted_user_id'].tolist()
    #saving other attributes (to assigne it to a nodes as property)
    verified_for_user[e]=u.df['author.verified']
    location_for_user[e]=u.df['author.location'].tolist()
   # name_for_user[e]=u.df['author.name']
   # username_for_user[e]=u.df['author.username']
    
    
    

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

...

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

In [15]:
user_id=[]
name=[]
hashtag=[]
retweet=[]
location=[]
verified=[]
for u,h in hashtag_for_user.items():
    user_id.append(u)
    hashtag.append(h)
for u,r in retweet_for_user.items():
    retweet.append(r)
for u,n in name_for_user.items():
    name.append(n)
for u,l in location_for_user.items():
    location.append(l)
for u,v in verified_for_user.items():
    verified.append(v)

In [20]:
print(len(verified))

77


In [28]:
#create a pandas dataframe
df_nodes=pd.DataFrame({"user_id":user_id,
                      'name': name,
                      'retweet': retweet,
                      "hashtag":hashtag,
                     'location':location,
                     'verified':verified})
df_nodes

,user_id,name,retweet,hashtag,location,verified
0,1357718765941710848,"0 PeterJohnKing4 Name: author.username, dty...",[1219232377605644289],"[[ukraine, russia]]",[Somewhere],"0 False Name: author.verified, dtype: bool"
1,1151202637582348294,0 hephaistos_ai 1 hephaistos_ai 2 ...,"[1246042405561872384, 1533187263156432896, 116...","[[covid, gouvernementdelahonte, nouvelordremon...","[Paris, Paris, Paris, Paris, Paris, Paris, Par...",0 False 1 False 2 False 3 Fals...
2,1466835011454189571,0 RightAndFree_ 5 RightAndFree_ 6 ...,"[1219232377605644289, 172377413, 1219232377605...","[[ukraine, russia], [ny19], [russia, ukraine],...","[nan, nan, nan, nan]",0 False 5 False 6 False 16 Fals...
3,4813333392,0 PansareKetan 8 PansareKetan 13 Pa...,"[81386285, 576533046, 849006404, 19897138]","[[shivsena, covid], [bjpjawabdo], [sanjaypande...","[nan, nan, nan, nan]",0 False 8 False 13 False 19 Fals...
4,2471461255,0 CanadaRomania 2 CanadaRomania 4 ...,"[121514648, 157981564, 14353202, 14353202, 410...","[[russia, ukraine, indonesia], [foodsecurity],...","[Bucharest, Romania, Bucharest, Romania, Bucha...",0 True 2 True 4 True 5 True 6 ...
...,...,...,...,...,...,...
72,47941897,0 ma000111 1 ma000111 2 ma000111 3...,"[47941897, 1167016822027112449, 47941897, 4794...","[[zelenskyy, warcrimes, donetsk, ukraine, ukra...","[England , England , England , England , Engla...",0 False 1 False 2 False 3 Fals...
73,2865524457,0 michelangela_K 2 michelangela_K 5 ...,"[23372297, 1364378809, 1091395730307190785, 33...","[[ukraine], [affenpocken, rki], [shinzoabe], [...","[Buxtehude, Buxtehude, Buxtehude, Buxtehude, B...",0 False 2 False 5 False 13 Fals...
74,1056303825995337730,1 MesarutF 4 MesarutF 12 MesarutF 1...,"[1496161188077621251, 1505481949854502915, 100...","[[donbas, ukraine, ukrainewarnews, stoprussia]...","[nan, nan, nan, nan, nan]",1 False 4 False 12 False 13 Fals...
75,3245630026,0 VerachikaV 2 VerachikaV 3 Verach...,"[191145819, 23372297, 1231514832479948802, 206...","[[forgottensoulshour, rescuedog, rehomehour, r...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]",0 False 2 False 3 False 4 Fals...


In [29]:
#convert nested lists on set
outer_list = [] 
def lists_to_list(nested_lists): 
    for el in nested_lists: 
        if type(el) == list: 
            lists_to_list(el) 
        else: 
            outer_list.append(el)
    return set(outer_list)

In [30]:
outer_list = [] 
df_nodes['hashtag']=df_nodes['hashtag'].map(lists_to_list)
outer_list = [] 
df_nodes['retweet']=df_nodes['retweet'].map(lists_to_list)
outer_list = [] 
df_nodes['name']=df_nodes['name'].map(lists_to_list)
outer_list = [] 
df_nodes['location']=df_nodes['location'].map(lists_to_list)
outer_list = [] 
df_nodes['verified']=df_nodes['verified'].map(lists_to_list)

In [45]:
boolean={'True','False'}
for b in df_nodes['verified']:
    print(b.intersection(boolean)) #=={'True','False'}:
       # b=='True'

set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


In [46]:
df_nodes

,user_id,name,retweet,hashtag,location,verified
0,1357718765941710848,{PeterJohnKing4},{1219232377605644289},"{ukraine, russia}",{Somewhere},{False}
1,1151202637582348294,"{PeterJohnKing4, hephaistos_ai}","{1533187263156432896, 24343635, 15422873631492...","{plandemic, staycovidsafe, vote, vacci, republ...","{Paris, Somewhere}",{False}
2,1466835011454189571,"{PeterJohnKing4, RightAndFree_, hephaistos_ai}","{1533187263156432896, 24343635, 15422873631492...","{plandemic, staycovidsafe, vote, trumpisatrait...","{Paris, nan, Somewhere, nan, nan, nan}",{False}
3,4813333392,"{PeterJohnKing4, RightAndFree_, PansareKetan, ...","{1533187263156432896, 24343635, 15422873631492...","{plandemic, staycovidsafe, vote, trumpisatrait...","{Paris, nan, nan, nan, Somewhere, nan, nan, na...",{False}
4,2471461255,"{PeterJohnKing4, CanadaRomania, RightAndFree_,...","{1533187263156432896, 33285177, 24343635, 4105...","{trumpisatraitor, republican, covidvaccine, tr...","{Bucharest, Romania, Paris, nan, nan, nan, Som...","{False, True}"
...,...,...,...,...,...,...
72,47941897,"{PeterJohnKing4, Douglas17569801, CanadaRomani...","{791612573531668481, 2932618967, 1342474419944...","{, buckwheat, nutrition, trumpisatraitor, majo...","{nan, DCEU Earth 666, MCU Earth 616, nan, Repu...","{False, True}"
73,2865524457,"{PeterJohnKing4, Douglas17569801, CanadaRomani...","{791612573531668481, 2932618967, 3315313691, 1...","{, buckwheat, nutrition, trumpisatraitor, majo...","{nan, DCEU Earth 666, MCU Earth 616, nan, Repu...","{False, True}"
74,1056303825995337730,"{PeterJohnKing4, Douglas17569801, CanadaRomani...","{791612573531668481, 2932618967, 3315313691, 1...","{, buckwheat, nutrition, trumpisatraitor, majo...","{nan, DCEU Earth 666, MCU Earth 616, nan, Repu...","{False, True}"
75,3245630026,"{PeterJohnKing4, Douglas17569801, CanadaRomani...","{791612573531668481, 2932618967, 3315313691, 1...","{, buckwheat, nutrition, trumpisatraitor, majo...","{nan, DCEU Earth 666, MCU Earth 616, nan, nan,...","{False, True}"


In [37]:
proukr=set(["slavaukraini"]) #1 cat
prorus=set(["istandwithputin", "istandwithrussia"]) #2 cat
pax=set(["stopwarinukraine"]) #3 cat
cat=set.union(proukr,prorus,pax)
count1=0
#count2=0
#count3=0
count4=0
index1=[]
#index2=[]
#index3=[]
index4=[]

for i, row in enumerate(df_IDK1['hashtag']): 
    if row.intersection(cat)==set():#no intersezione   
        #print(row.intersectionn(cat))
        #print("l'utente {}  appartiene alla 4 categoria".format(row.intersection(cat)))
        #print("4 categoria", row)
        count4 += 1
        index4.append(i)
        
    else: #c'è intersezione: uno della 3 categorie
        #print("non è la 4 categoria perchè contiene",row.intersection(cat))
       # if row.intersection(proukr)!=set(): #c'è intersez con proukr
           # count1 += 1
            index1.append(i) #aggiungo alle rige da tgliere
            print("l'utente {} non  appartiene alla 4 categoria perchè usa {}".format(i,row.intersection(cat)))
       # elif row.intersection(prorus)!=set():
          #  count2 += 1
           # index2.append(i)
        #elif row.intersection(pax)!=set():
           # count3 += 1
           #index3.append(i)

l'utente 28 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 29 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 30 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 31 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 32 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 33 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 34 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 35 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 36 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 37 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 38 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 39 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 40 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 41 non  appartiene alla 4 categoria perchè

In [29]:
print('proukr = {} \n'
     'prourus = {} \n'
     'pax = {} \n'
     'nocare = {} '.format(count1,count2,count3,count4))

proukr = 391 
prourus = 0 
pax = 0 
nocare = 28 


## 3 parte categorizzazione tweet e utenti di proukr, prorus, pax

## 4 parte: edge tra utenti nel dataset

In [67]:
df_IDK1['retweet']

0     {1498176452084256772, 1219232377605644289, 135...
1     {1359617172033724416, 14106476, 14981764520842...
2     {1521787270260719616, 923231030160400384, 1359...
3     {1521787270260719616, 923231030160400384, 1359...
4     {1521787270260719616, 923231030160400384, 1359...
                            ...                        
73    {720139699, 14106476, 1458038317619073044, 316...
74    {720139699, 14106476, 1458038317619073044, 316...
75    {720139699, 14106476, 1458038317619073044, 316...
76    {720139699, 14106476, 1458038317619073044, 316...
77    {720139699, 14106476, 1458038317619073044, 316...
Name: retweet, Length: 78, dtype: object